# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [12]:
from pyspark.sql import SparkSession


# TODOS: 
# 1) import any other libraries you might need
from pyspark.sql.functions import isnan, count, when, col, desc, udf, sort_array, asc, avg
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType
# 2) instantiate a Spark session 
spark = SparkSession.builder.appName('DataFrame_quiz').getOrCreate()
# 3) read in the data set located at the path "data/sparkify_log_small.json"
path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)
# 4) write code to answer the quiz questions 

# Question 1

Which page did user id "" (empty string) NOT visit?

In [24]:
# TODO: write your code to answer question 1
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [16]:
blank_pages = user_log.select("page").where(user_log.userId=="").dropDuplicates()

In [18]:
blank_pages.show()

+-----+
| page|
+-----+
| Home|
|About|
|Login|
| Help|
+-----+



In [13]:
blank_pages = user_log.filter(user_log.userId == '') \
    .select(col('page').alias('blank_pages')) \
    .dropDuplicates()

In [15]:
blank_pages.show()

+-----------+
|blank_pages|
+-----------+
|       Home|
|      About|
|      Login|
|       Help|
+-----------+



In [5]:
all_pages=user_log.select("page").dropDuplicates()

In [20]:
all_pages.show()

+----------------+
|            page|
+----------------+
|Submit Downgrade|
|            Home|
|       Downgrade|
|          Logout|
|   Save Settings|
|           About|
|        Settings|
|           Login|
|        NextSong|
|            Help|
|         Upgrade|
|           Error|
|  Submit Upgrade|
+----------------+



In [23]:
for row in set(all_pages.collect())-set(blank_pages.collect()):
    print(row)

Row(page='Downgrade')
Row(page='Error')
Row(page='Upgrade')
Row(page='NextSong')
Row(page='Logout')
Row(page='Settings')
Row(page='Save Settings')
Row(page='Submit Downgrade')
Row(page='Submit Upgrade')


# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [30]:
# TODO: use this space to explore the behavior of the user with an empty string
user_log.filter(user_log.userId=="").select(["gender","registration"]).dropDuplicates().collect()

[Row(gender=None, registration=None)]

# Question 3

How many female users do we have in the data set?

In [38]:
user_log.show(n=1)

+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|       artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|Showaddywaddy|Logged In|  Kenneth|     M|          112|Matthews|232.93342| paid|Charlotte-Concord...|   PUT|NextSong|1509380319284|     5132|Christmas Tears W...|   200|1513720872284|"Mozilla/5.0 (Win...|  1046|
+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+------

In [37]:
# TODO: write your code to answer question 3
user_log.filter(user_log.gender=='F').select("gender","userId").dropDuplicates().count()


462

# Question 4

How many songs were played from the most played artist?

In [46]:
# TODO: write your code to answer question 4
user_log.filter(user_log.page == 'NextSong').select('Artist') \
    .groupBy('Artist') \
    .agg({'Artist':'count'}) \
    .withColumnRenamed('count(Artist)', 'Artistcount') \
    .sort(desc('Artistcount')) \
    .show(1)

+--------+-----------+
|  Artist|Artistcount|
+--------+-----------+
|Coldplay|         83|
+--------+-----------+
only showing top 1 row



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [48]:
# TODO: write your code to answer question 5
# TODO: filter out 0 sum and max sum to get more exact answer

function = udf(lambda ishome : int(ishome == 'Home'), IntegerType())

user_window = Window \
    .partitionBy('userID') \
    .orderBy(desc('ts')) \
    .rangeBetween(Window.unboundedPreceding, 0)

cusum = user_log.filter((user_log.page == 'NextSong') | (user_log.page == 'Home')) \
    .select('userID', 'page', 'ts') \
    .withColumn('homevisit', function(col('page'))) \
    .withColumn('period', Fsum('homevisit').over(user_window))

cusum.filter((cusum.page == 'NextSong')) \
    .groupBy('userID', 'period') \
    .agg({'period':'count'}) \
    .agg({'count(period)':'avg'}).show()

+------------------+
|avg(count(period))|
+------------------+
| 6.898347107438017|
+------------------+

